In [ ]:
"""
C4.5 Binary Decision Tree Implementation

Usage: 
Read csv file in; will be stored as a 2 Dimensional list. (See fread())
Train a classifier (i.e. train(list))
Prune the decision tree (i.e. prune_tree(tree, 0.5))
Predict the result (i.e. predict([.....], classifier))

The function assumes that the last column of your data is populated by labels.
"""

In [16]:
#from sklearn.feature_selection import mutual_info_classif
from collections import OrderedDict
from math import log
import csv
import timeit

In [30]:
df = fread("./Iris.csv", True)
#n * d data, n rows d cols
df = [i[1:] for i in df]

#these are for performance testing estimates
df2 = df[:int(len(df)/2)] #n/2
df3 = [i[:2] for i in df] #d/2
df4 = [i[:-1]+i for i in df] #2d
df5 = df+df # 2n
df6 = df5+df5 #4n
df7 = [i[:-1]+i for i in df4] #4d

In [2]:
def n_distinct_dict(rows):
    counts = dict()
    for x in rows:
        xs = x[-1]
        if xs not in counts: 
            #Add it to the counts dictionary
            counts[xs] = 0
        counts[xs] += 1
    return counts

def entropy(X):
    """
    Calculate Entropy (as per Octavian)
    """
    counts = n_distinct_dict(X)
    log_2 = lambda x: log(x)/log(2)
    #Declare entropy value
    entropy = 0.0
    
    for c in counts:
        #Calculate P(C_i)
        p = float(counts[c])/len(X)
        entropy = entropy -  p*log_2(p)
    return entropy

def gini(X):
    """
    Calculate Gini Index
    """
    total = len(X)
    counts = n_distinct_dict(X)
    imp = 0.0
    
    for k1 in counts:
        p1 = float(counts[k1])/total  
        for k2 in counts:
            if k1 == k2: continue
            p2 = float(counts[k2])/total
            imp += p1*p2
    return imp


In [27]:
print(test)

{'Test1': 0    1
1    2
2    3
3    1
Name: Test1, dtype: int64, 'Test2': 0    1
1    2
2    3
3    1
Name: Test2, dtype: int64}


In [3]:
class Tree:
    """
    Decision Tree class
    """
    def __init__(self, col=-1, value=None, right_branch=None, left_branch=None, results=None):
        self.col = col
        self.value = value
        self.right_branch = right_branch
        self.left_branch = left_branch
        self.results = results


def prune_tree(tree, least_gain, eval_fun = entropy):
    """
    tree : type Tree
    eval_fun : entropy(X) or gini(X)
    least_gain : float
    """
    
    if tree.right_branch.results == None: #if the right branch is a node
        prune_tree(tree.right_branch, least_gain, eval_fun)
    if tree.left_branch.results == None: #if the left branch is a node
        prune_tree(tree.left_branch, least_gain, eval_fun)
    if (tree.right_branch.results != None) and (tree.left_branch.results != None):
        right, left = [], []
        for v, c in tree.right_branch.results.items(): 
            right += [[v]] * c
        for v, c in tree.left_branch.results.items(): 
            left += [[v]] * c
        p = float(len(right)) / len(left + right)
        diff_entropy = eval_fun(right+left) - p*eval_fun(right) - (1-p)*eval_fun(left)
        if diff_entropy < least_gain:
            tree.right_branch, tree.left_branch = None, None
            tree.results = n_distinct_dict(left + right)

In [9]:
"""
Helper functions
"""
def type_conversion(val):
        val = val.strip()
        
        try:
            if '.' in val:
                return float(val)
            else:
                return int(val)
            
        except ValueError:
            #For other types, return
            return val
        
def fread(f, col_labels = False):
    """
    takes a filepath, f, and a boolean argument, col_labels.
    By default, col_labels is False, implying that the columns do not have labels. If set to true,
    fread will remove the row containing column labels at the end.
    """
    data = csv.reader(open(f, 'rt'))
    lst = [[type_conversion(i) for i in r] for r in data]
    if col_labels:
        lst.pop(0)
    return lst

In [4]:
def partition(r, c, val):
    """
    Function to partition the data based on value
    """
    #Declare anonymous function
    split_fun = None
    if isinstance(val, float) or isinstance(val, int): 
        #Anonymous function for numeric values
        split_fun = lambda row : row[c] >= val
    else: 
        #For string values
        split_fun = lambda row : row[c] == val
    list1 = [row for row in r if split_fun(row)]
    list2 = [row for row in r if not split_fun(row)]
    return (list1, list2)



def train(lst, criteria = entropy):
    """
    Decision tree construction - by default, the entropy function is used to calculate the criteria for splitting. 
    lst : dataframe with the last column reserved for labels
    criteria : entropy or gini calculation function
    """
    #Base Case: Empty Set
    if len(lst) == 0: 
        return Tree()
    
    #Calculate Entropy/Gini of current X, declare A_best, create sets/gain accordingly
    score = criteria(lst)
    Attribute_best = None
    Set_best = None
    Gain_best = 0.0
    

    num_col = len(lst[0]) - 1  # last column of lst is labels
    for c in range(num_col):
        col_val = [row[c] for row in lst]
        for value in col_val:
            #Split dataset
            (set1, set2) = partition(lst, c, value)
            # Calculate Gain
            p = float(len(set1))/len(lst)
            gain = score - p*criteria(set1) - (1-p)*criteria(set2)
            if gain>Gain_best and len(set1)>0 and len(set2)>0:
                Gain_best = gain
                Attribute_best = (c, value)
                Set_best = (set1, set2)

    if Gain_best > 0:
        #Recursive Call on partitioned Sets
        right_branch = train(Set_best[0])
        left_branch = train(Set_best[1])
        return Tree(col=Attribute_best[0], value=Attribute_best[1], right_branch=right_branch, left_branch=left_branch)
    
    else:
        return Tree(results=n_distinct_dict(lst))


def tree_classify(X, tree):
    """
    Classification function using a list read from fread, X, and grown Decision Tree, tree
    """
    if tree.results != None:
        return (tree.results)
    else:
        b = None
        val = X[tree.col] #Retrieve label from dataframe X
        if isinstance(val, float) or isinstance(val,int):
            #Traversing decision tree for numerics
            if val >= tree.value:
                branch = tree.right_branch
            else:
                branch = tree.left_branch
            
        else:
            #Traversing decision tree for non-numeric types
            if val == tree.value:
                branch = tree.right_branch
            else:
                branch = tree.left_branch
    return tree_classify(X, branch)

def predict(x, classifier):
    return list(tree_classify(x, classifier))[0]

In [157]:
"""
TODO:
Optimize
Testing
"""

data = fread("./test_val_dump.csv", True)
drop_first_col = []
for x in data:
    drop_first_col.append(x[1:])

In [159]:
tree = train(drop_first_col)
prune_tree(tree, 0.5)

In [160]:
print(predict([2,0,2,6,1,0,0,3,1,0,0,2,0,0,0,0,0,1,.223606798,0,.285714,.141421,0,.253546],tree))

1


In [15]:
"""
The base cases are the following:

•  All the examples from the training set belong to the same class ( a tree leaf labeled with that class is returned ).

•  The training set is empty ( returns a tree leaf called failure ).

•  The attribute list is empty ( returns a leaf labeled with the most frequent class or the disjuction of all the classes).
https://octaviansima.wordpress.com/2011/03/25/decision-trees-c4-5/
"""

'\nThe base cases are the following:\n\n•  All the examples from the training set belong to the same class ( a tree leaf labeled with that class is returned ).\n\n•  The training set is empty ( returns a tree leaf called failure ).\n\n•  The attribute list is empty ( returns a leaf labeled with the most frequent class or the disjuction of all the classes).\nhttps://octaviansima.wordpress.com/2011/03/25/decision-trees-c4-5/\n'

In [19]:
#n * d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.09544761092518456


In [20]:
#n/2 * d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df2)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.01982154435128905


In [21]:
#n * d/2
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df3)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.038575201586354524


In [24]:
#n * 2d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df4)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.1923355341120623


In [25]:
#2n * d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df5)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.3688689094118308


In [29]:
#4n * d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df6)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

1.384299785438925


In [31]:
#n * 4d
times = []
for i in range(25):
    start = timeit.default_timer()
    tree = train(df7)
    prune_tree(tree, 0.5)
    stop = timeit.default_timer()
    times.append(stop-start)
print(sum(times)/len(times)) 

0.3654549332521856
